In [ ]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import gym
from gym import spaces
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Input
from tensorflow.keras.optimizers import Adam
import random

# Memuat dataset
data_path = 'final_hcc_data_cleaned.csv'  # Update path sesuai lokasi dataset Anda
data = pd.read_csv(data_path)

# Pastikan kolom target ada di dataset
target_column = 'treatment_success'
if target_column not in data.columns:
    raise ValueError(f"Kolom '{target_column}' tidak ditemukan dalam dataset.")

# Memisahkan fitur dan target
features = data.drop(target_column, axis=1)
target = data[target_column]

# Normalisasi fitur
scaler = StandardScaler()
features_scaled = scaler.fit_transform(features)

# Memisahkan data menjadi data latih dan data uji
X_train, X_test, y_train, y_test = train_test_split(features_scaled, target, test_size=0.2, random_state=42)

# Membuat lingkungan simulasi untuk terapi medis
class MedicalTreatmentEnv(gym.Env):
    def __init__(self, data, labels):
        super(MedicalTreatmentEnv, self).__init__()
        self.data = data
        self.labels = labels
        self.action_space = spaces.Discrete(2)  # 0 untuk terapi konservatif, 1 untuk terapi agresif
        self.observation_space = spaces.Box(low=-np.inf, high=np.inf, shape=(data.shape[1],), dtype=np.float32)
        self.current_step = 0

    def step(self, action):
        self.current_step += 1
        done = self.current_step == len(self.labels)
        reward = 1 if self.labels[self.current_step] == action else -1
        next_state = self.data[self.current_step] if not done else np.zeros(self.data.shape[1])
        return next_state, reward, done, {}

    def reset(self):
        self.current_step = 0
        return self.data[self.current_step]

# Model DQN untuk memilih terapi
class DQNAgent:
    def __init__(self, state_size, action_size):
        self.state_size = state_size
        self.action_size = action_size
        self.memory = []
        self.gamma = 0.95  # Faktor diskon
        self.epsilon = 1.0  # Tingkat eksplorasi
        self.epsilon_min = 0.01
        self.epsilon_decay = 0.995
        self.model = self.build_model()

    def build_model(self):
        model = Sequential([
            Input(shape=(self.state_size,)),
            Dense(64, activation='relu'),
            Dense(64, activation='relu'),
            Dense(self.action_size, activation='linear')
        ])
        model.compile(loss='mse', optimizer=Adam(learning_rate=0.001))
        return model

    def act(self, state):
        if np.random.rand() <= self.epsilon:
            return random.randrange(self.action_size)
        act_values = self.model.predict(state)
        return np.argmax(act_values[0])

# Inisialisasi lingkungan dan agen
env = MedicalTreatmentEnv(X_train, y_train)
agent = DQNAgent(env.observation_space.shape[0], env.action_space.n)

# Menyimpan model untuk evaluasi
agent.model.save('hcc_treatment_dqn_model.keras')

# Menampilkan hasil per pasien
print("Hasil Prediksi Terapi Per Pasien:")
for i, patient in enumerate(X_test):
    predicted_action = agent.act(patient.reshape(1, -1))
    actual_outcome = 'Sukses' if y_test.iloc[i] == 1 else 'Gagal'
    treatment_type = 'Agresif' if predicted_action == 1 else 'Konservatif'
    specific_treatment = "Operasi, Ablasi, atau Kemoterapi" if treatment_type == 'Agresif' else "Obat Sorafenib atau Terapi Pendukung"
    print(f"Pasien {i + 1}: Terapi yang direkomendasikan: {treatment_type}, Terapi Spesifik: {specific_treatment}, Hasil aktual: {actual_outcome}")
